In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from tqdm import tqdm

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
def calc_acc(preds,labels):
  _,pred_max=torch.max(preds,1)
  acc=torch.sum(pred_max==labels.data,dtype=torch.float64)/len(preds)
  return acc

In [35]:

class Model(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1=nn.Conv2d(3,512,(3,3),(1,1),(0,0))
    self.conv2=nn.Conv2d(512,128,(3,3),(1,1),(0,0))
    self.conv3=nn.Conv2d(128,64,(3,3),(1,1),(1,1))
    self.conv4=nn.Conv2d(64,32,(3,3),(1,1),(1,1))
    self.conv5=nn.Conv2d(32,32,(3,3),(1,1),(1,1))

    self.fc1=nn.Linear(32*2*2,256)
    self.fc2=nn.Linear(256,512)
    self.fc3=nn.Linear(512,10)
  
  def forward(self,x):
    x=F.relu(self.conv1(x))
    x=F.max_pool2d(x,kernel_size=(2,2))

    x=F.relu(self.conv2(x))
    x=F.max_pool2d(x,kernel_size=(2,2))

    x=F.relu(self.conv3(x))
    x=F.max_pool2d(x,kernel_size=(2,2))

    x=F.relu(self.conv4(x))
    x=F.max_pool2d(x,kernel_size=(2,2))

    x=F.relu(self.conv5(x))

    x=torch.flatten(x,start_dim=1)

    x=F.relu(self.fc1(x))
    x=F.relu(self.fc2(x))
    x=self.fc3(x)
    x=torch.softmax(x,dim=1)
    return x

In [36]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=Model().to(device)

In [37]:
#hyper parameters
batch_size=16
epochs=40
lr=0.0001


In [38]:
transform=transforms.Compose([
                                   transforms.RandomRotation(10),
                                   transforms.Resize((50,50)),
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))

])

dataset=torchvision.datasets.ImageFolder(root="/content/drive/MyDrive/persian-mnist",transform=transform)
train_data_loader=torch.utils.data.DataLoader(dataset,batch_size=batch_size,shuffle=True)

In [39]:
#compile 
optimizer=torch.optim.Adam(model.parameters(),lr=lr)
loss_function=nn.CrossEntropyLoss()

In [ ]:
model.train()
for epoch in range(epochs):
  train_loss=0.0
  train_acc=0.0
  for images,labels in tqdm(train_data_loader):
    images,labels=images.to(device),labels.to(device)
    optimizer.zero_grad()

    preds=model(images)
    loss=loss_function(preds,labels)
    loss.backward()
    optimizer.step()
    train_loss+=loss
    train_acc+=calc_acc(preds,labels)
  
  total_loss=train_loss/len(train_data_loader)
  total_acc=train_acc/len(train_data_loader)
  print(f"epochs: {epoch} , loss: {total_loss}, acc: {total_acc}")

In [41]:
torch.save(model.state_dict(),"persian.pth")